### Uplift modeling

Uplift modeling: http://people.cs.pitt.edu/~milos/icml_clinicaldata_2012/Papers/Oral_Jaroszewitz_ICML_Clinical_2012.pdf  
  
dataset: https://ailab.criteo.com/criteo-uplift-prediction-dataset/
- f0, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11: feature values (dense, float)
- treatment: treatment group (1 = treated, 0 = control)
- conversion: whether a conversion occured for this user (binary, label)
- visit: whether a visit occured for this user (binary, label)
- exposure: treatment effect, whether the user has been effectively exposed (binary)  
  
What would I do:  
Target conversion to improve  
やりたいこと：  
コンバージョン率を上げるため、実施したABテストの結果を検討

### First model: 2 model approach

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
data = pd.read_csv('data/criteo-uplift.csv')
data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
0,1.991981,3.263641,9.241052,3.735871,3.506733,10.161281,2.981721,-0.166689,-0.584392,9.850093,-1.8609,4.157648,1,0,0,0
1,1.991981,3.263641,9.274430,3.735871,3.506733,10.161281,2.981721,-0.166689,-0.584392,9.850093,-1.8609,4.157648,1,0,0,0
2,1.991981,3.263641,9.226241,3.735871,3.506733,10.161281,2.981721,-0.166689,-0.584392,9.850093,-1.8609,4.157648,1,0,0,0
3,1.991981,3.263641,9.274572,3.735871,3.506733,10.161281,2.981721,-0.166689,-0.584392,9.850093,-1.8609,4.157648,1,0,0,0
4,1.991981,3.263641,9.332619,3.735871,3.506733,10.161281,2.981721,-0.166689,-0.584392,9.850093,-1.8609,4.157648,0,0,0,0


In [4]:
# data divide treatment
treatment_df = data[data['treatment'] == 1]
control_df = data[data['treatment'] == 0]

In [5]:
# check dataset size
print(treatment_df.shape)
print(control_df.shape)

(21408827, 16)
(3900655, 16)


In [6]:
# data devide train and test
X_control_train, X_control_test, y_control_train, y_control_test = train_test_split(control_df[['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']], 
                                                                                    control_df['conversion'], random_state=74)

In [ ]:
X_treatment_train, X_treatment_test, y_treatment_train, y_treatment_test = train_test_split(treatment_df[['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']], 
                                                                                            treatment_df['conversion'], random_state=74, test_size=len(y_control_test))

In [ ]:
# treatment model 
treatment_model = LogisticRegression()
treatment_model.fit(X_treatment_train, y_treatment_train)

In [ ]:
for column, coef in zip(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11'], treatment_model.coef_[0]):
    print(f'{column}: {coef}')

In [ ]:
# control model
control_model = LogisticRegression()
control_model.fit(X_control_train, y_control_train)

In [ ]:
for column, coef in zip(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11'], control_model.coef_[0]):
    print(f'{column}: {coef}')

TODO: 英語でここまでの結論まとめる  
  
  
施策の有無に左右されず、f4とf11がコンバージョンに寄与している模様

In [ ]:
result_df = pd.DataFrame()
result_df['treatment'] = treatment_model.predict_proba(X_treatment_test).flatten()
result_df['control']= control_model.predict_proba(X_control_test).flatten()
result_df['uplift'] = result_df['treatment'] / result_df['control']
result_df

In [ ]:
result_df.describe()

### 1 model approach

In [ ]:
# 